# Finding Otherside

_Though I still want [Pigstep](https://github.com/OpenBagTwo/AngerLocalBeehive/blob/main/notebooks/Pigstep.ipynb)._

I'm looting ancient cities, and although I know that I won't be able to find out in advance just which chests have "Otherside," I can at leas find out if there are any I've missed.

## Imports, Setup and Macros

In [1]:
import json
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict, Set

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size = size / 1024
    return f"{size} TB"

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (8.6 KB)
- level.dat (13.6 KB)
- level.dat_old (13.6 KB)
- level11326231134829879582.dat (0.0 B)
- level14927678268100607923.dat (0.0 B)
- level1786655981796876926.dat (0.0 B)
- level4463453738642305340.dat (0.0 B)
- level6425531070021529407.dat (0.0 B)
- level7443636089696258371.dat (0.0 B)
- level8832581565660323154.dat (0.0 B)
- playerdata (folder)
- poi (folder)
- region (folder)
- serverconfig (folder)
- session.lock (3.0 B)
- stats (folder)


## Chest Search 1.19

From the [Pigstep](https://github.com/OpenBagTwo/AngerLocalBeehive/blob/main/notebooks/Pigstep.ipynb) notebook, I know where to find chest data and what I can and can't expect to find. To wit: I can't find what's in a loot chest, but I can find out what loot table it's using.

In [5]:
all_region_files = sorted(
    (save_folder / "region").glob("*"), key=lambda path: -path.stat().st_size
)
for path in all_region_files[:10]:
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")
print(f"... {len(all_region_files) - 10} more")

- r.-5.-1.mca (113.2 MB)
- r.-5.0.mca (27.4 MB)
- r.-5.-2.mca (12.3 MB)
- r.-2.-4.mca (12.0 MB)
- r.-2.1.mca (11.8 MB)
- r.-3.1.mca (11.7 MB)
- r.8.-3.mca (11.5 MB)
- r.-2.-12.mca (11.3 MB)
- r.-1.-13.mca (11.3 MB)
- r.-3.0.mca (11.3 MB)
... 264 more


### Getting The Distinct Loot Tables

I'm sure I could find this in some sort of datapack tutorial, but nothing beats parsing it fresh.

In [6]:
%%time
unique_loot_tables = []
for path in all_region_files:
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["block_entities"]:
            if entity["id"].value != "minecraft:chest":
                continue
            if "LootTable" not in entity:
                continue
            loot_table = entity["LootTable"].value
            if loot_table not in unique_loot_tables:
                print(f"Found a new loot table:\n - {loot_table}")
                unique_loot_tables.append(loot_table)

Found a new loot table:
 - minecraft:chests/simple_dungeon
Found a new loot table:
 - minecraft:chests/igloo_chest
Found a new loot table:
 - minecraft:chests/village/village_plains_house
Found a new loot table:
 - minecraft:chests/buried_treasure
Found a new loot table:
 - minecraft:chests/ruined_portal
Found a new loot table:
 - minecraft:chests/shipwreck_treasure
Found a new loot table:
 - minecraft:chests/shipwreck_map
Found a new loot table:
 - minecraft:chests/underwater_ruin_small
Found a new loot table:
 - minecraft:chests/village/village_taiga_house
Found a new loot table:
 - minecraft:chests/village/village_cartographer
Found a new loot table:
 - minecraft:chests/village/village_weaponsmith
Found a new loot table:
 - minecraft:chests/village/village_toolsmith
Found a new loot table:
 - minecraft:chests/shipwreck_supply
Found a new loot table:
 - minecraft:chests/village/village_snowy_house
Found a new loot table:
 - minecraft:chests/village/village_tannery
Found a new loot ta

... it actually took less time to [just pull up the wiki...](https://minecraft.fandom.com/wiki/Loot_table#List_of_vanilla_loot_tables).

Cross-referencing the [Music Discs page](https://minecraft.fandom.com/wiki/Music_Disc#Discs) I know the ones I need to look afort are:

- `minecraft:chests/ancient_city`
- `minecraft:chests/stronghold_corridor`

And if I'd had any idea how long the above search would take I would have grabbed them during the first-go-though. Oh well...

In [7]:
from concurrent.futures import ProcessPoolExecutor, as_completed

In [8]:
def find_chests(path: Path) -> None:
    """Print the coordinates of any matching
    chest in the region file specified by the path

    Parameters
    ----------
    path : Path
        The region file path

    Returns
    -------
    None
        Just prints to stdout

    Notes
    -----
    Not best practice but also not
    super worried about prints running
    into each other
    """
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["block_entities"]:
            if entity["id"].value != "minecraft:chest":
                continue
            if "LootTable" not in entity:
                continue
            loot_table = entity["LootTable"].value
            if loot_table not in (
                "minecraft:chests/stronghold_corridor",
                "minecraft:chests/ancient_city",
            ):
                continue
            coordinates = "({}, {}. {})".format(
                *[entity[coord].value for coord in "xyz"]
            )
            print(f"Found {loot_table[17:]} chest at {coordinates}")

Ooh, it's been a while since I've had a reason to use `concurrent.futures`...

In [9]:
%%time
with ProcessPoolExecutor(max_workers=7) as executor:
    futures = []
    for region_file_path in all_region_files:
        futures.append(executor.submit(find_chests, region_file_path))
    for result in as_completed(futures):
        pass

Found stronghold_corridor chest at (686, -48. -4612)
Found ancient_city chest at (-725, -37. -6070)
Found ancient_city chest at (-720, -50. -6137)
Found ancient_city chest at (-705, -50. -6037)
Found ancient_city chest at (-703, -49. -6112)
Found ancient_city chest at (-687, -50. -6063)
Found ancient_city chest at (-670, -44. -6066)
Found ancient_city chest at (-614, -37. -6070)
Found ancient_city chest at (-595, -49. -6078)
Found ancient_city chest at (-605, -50. -6036)
Found ancient_city chest at (-596, -50. -6023)
Found ancient_city chest at (-597, -50. -6032)
Found ancient_city chest at (-578, -50. -6110)
Found ancient_city chest at (-564, -48. -6138)
Found ancient_city chest at (-557, -49. -6102)
Found stronghold_corridor chest at (2560, -22. 920)
Found stronghold_corridor chest at (670, -46. -4567)
Found stronghold_corridor chest at (702, -42. -4600)
Found stronghold_corridor chest at (2525, -22. 916)
Found stronghold_corridor chest at (2544, -24. 972)
Found ancient_city chest at

I was a little worried that disk IO was going to turn out to be the bottleneck, but actually each process hit 100% CPU utilization (and my core temps got nice and toasty).

Anyway, that should give me plenty of chests to search (the odds of finding Otherside are about 1 in 12).

## Pigstep?

Sure. Let's search `DIM-1`.

In [10]:
def find_chests(path: Path) -> None:
    """Print the coordinates of any matching
    chest in the region file specified by the path

    Parameters
    ----------
    path : Path
        The region file path

    Returns
    -------
    None
        Just prints to stdout

    Notes
    -----
    Not best practice but also not
    super worried about prints running
    into each other
    """
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["block_entities"]:
            if entity["id"].value != "minecraft:chest":
                continue
            if "LootTable" not in entity:
                continue
            loot_table = entity["LootTable"].value
            if loot_table != "minecraft:chests/bastion_other":
                continue
            coordinates = "({}, {}. {})".format(
                *[entity[coord].value for coord in "xyz"]
            )
            print(f"Found a potential pigstep chest at {coordinates}")

In [12]:
%%time
with ProcessPoolExecutor(max_workers=7) as executor:
    futures = []
    for region_file_path in (save_folder / "DIM-1" / "region").glob("*"):
        futures.append(executor.submit(find_chests, region_file_path))
    for result in as_completed(futures):
        pass

Found a potential pigstep chest at (-940, 58. -1108)
Found a potential pigstep chest at (-915, 35. -1116)
Found a potential pigstep chest at (-915, 58. -1116)
Found a potential pigstep chest at (-919, 68. -1107)
Found a potential pigstep chest at (-555, 72. 536)
Found a potential pigstep chest at (-555, 72. 535)
Found a potential pigstep chest at (-555, 72. 539)
Found a potential pigstep chest at (-557, 68. 553)
Found a potential pigstep chest at (-548, 58. 557)
Found a potential pigstep chest at (-548, 35. 557)
Found a potential pigstep chest at (-555, 72. 571)
Found a potential pigstep chest at (-555, 72. 567)
Found a potential pigstep chest at (-555, 72. 568)
Found a potential pigstep chest at (-906, 38. -1111)
Found a potential pigstep chest at (-899, 58. -1114)
Found a potential pigstep chest at (-903, 68. -1105)
Found a potential pigstep chest at (-215, 40. -1540)
Found a potential pigstep chest at (-203, 40. -1539)
Found a potential pigstep chest at (-191, 40. -1544)
Found a pot

Well that's a lot more than I expected to find.